<a href="https://colab.research.google.com/github/Ihsan101/FreeCodeCamp-final-projects/blob/main/SMS%20Spam%20detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
df_train = pd.read_csv(train_file_path, sep="\t",header = None, names=['type', 'text'])
df_test = pd.read_csv(test_file_path, sep="\t",header = None, names=['type', 'text'])

df_train['type'] = df_train['type'].map({'spam': 1, 'ham': 0})
df_test['type'] = df_test['type'].map({'spam': 1, 'ham': 0})
df_test.head(255)

y_train = df_train.pop('type')
y_test = df_test.pop('type')

train_data = tf.data.Dataset.from_tensor_slices((df_train, y_train))
test_data = tf.data.Dataset.from_tensor_slices((df_test, y_test))

print(train_data)
encoder = keras.layers.TextVectorization()
encoder.adapt(train_data.map(lambda text, label: text))
maxlen =len(encoder.get_vocabulary())


model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(input_dim= maxlen, output_dim = 64, mask_zero = True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer = 'rmsprop', metrics = ['acc'])
history = model.fit(df_train, y_train, epochs = 8, validation_data = (df_test, y_test))



In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  value = model.predict([pred_text])
  value = value[0][0]
  if value <0.5:
    prediction = [value,"ham"]
  else:
    prediction = [value, "spam"]

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
